**Before running this example, install the required packages:**

In [37]:
!rm -rf ./*

In [38]:
# Install nuclab from TestPyPI, curie, and xlsxwriter
!pip install -i https://test.pypi.org/simple/ nuclab==0.0.5
!pip install -q xlsxwriter

Looking in indexes: https://test.pypi.org/simple/


## **Notebook Overview**

In this notebook, we demonstrate how to use **nuclab’s** `Yield` class to calculate theoretical end-of-bombardment (EoB) activity values. The workflow accounts for multiple possible reaction pathways, providing a flexible framework for modeling isotope production. The approach involves decomposing the target into thin slices, each thin enough to degrade the ion beam by an energy specificed by the user. Activity is calculated slice-by-slice, using each slice’s atomic areal density, the entrance energy of ions into that slice, and the relevant cross section at that energy. The total yield is then obtained by summing the contributions across all slices.

## Import Dependencies

- We use the ``Yield ``class from **nuclab's** ``production`` module.

In [39]:
import nuclab
from nuclab.production import Yield

# Standard data handling tools
import pandas as pd
import numpy as np

# To download and extract example data from GitHub
import os, urllib.request, zipfile

## Extract Example Data from Project Repository

This code block retrieves the required cross-section and SRIM stopping-power datasets from the project repository. The files are unpacked into the `example_data` directory, which serves as a ready-to-use data source for running the workflow described above.


In [40]:
# Make local dir
os.makedirs("data", exist_ok=True)

# Direct URL to example_data.zip in your repo
zip_url = "https://github.com/vivektara24/Separation-of-terbium-from-proton-irradiated-gadolinium-oxide-targets/raw/main/example_data.zip"
zip_path = "example_data.zip"

# Download ZIP
urllib.request.urlretrieve(zip_url, zip_path)

# Extract contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(".")

# Path to extracted folder
data_dir = "example_data"
print("Files in example_data:", os.listdir(data_dir))

Files in example_data: ['yield', 'cal-to-ser']


# Theoretical Yield Calculation Workflow

In [41]:
# Define directory where outputs of the nuclab package will be saved
# Will be created automatically in workflow
output_dir = 'outputs'

In [42]:
# Define the isotope_half_lives dictionary. Maps isotopes to half-lives in (s)
ISOTOPE_HALF_LIVES = {
    "153Tb": 2.34 * 24 * 60 * 60,
    "154Tb": 21.5 * 60 * 60,
    "154m1Tb": 9.994 * 60 * 60,
    "154m2Tb": 22.7 * 60 * 60,
    "155Tb": 5.32 * 24 * 60 * 60,
    "156Tb": 5.35 * 24 * 60 * 60,
    "156m2Tb": 5.3 * 60 * 60
}

In [43]:
# Calculation Parameters
DE = 0.001                # Energy slice width for target decomposition (MeV)

# Irradiation parameters
E0 = 12.6                 # Incident ion energy (MeV)
PROJECTILE_INTENSITY = 2.37e13  # Ion beam intensity (particles/s)
T_IRRAD = 1800            # Irradiation time (s)

# Target parameters
TARGET_THICKNESS = 0.037  # Target thickness (cm)
DENSITY = 3.385           # Target material density (g/cm³)
MOLECULAR_WEIGHT = 362.50 # Target material molecular weight (g/mol)

In [44]:
# Import SRIM simulation data for Gd2O3;
# extract ion energies (MeV) and corresponding projected ranges (cm)
df = pd.read_csv('/content/example_data/yield/Gd2O3-SRIM-DATA.csv')
SRIM_ENERGIES, SRIM_RANGES = df.iloc[:, 0], df.iloc[:, 1]

In [45]:
SRIM_ENERGIES

,Energy (MeV)
0,0.010
1,0.011
2,0.012
3,0.013
4,0.014
...,...
80,12.000
81,13.000
82,14.000
83,15.000


In [46]:
SRIM_RANGES

,Projected Range (cm)
0,0.000017
1,0.000019
2,0.000020
3,0.000022
4,0.000023
...,...
80,0.104000
81,0.119000
82,0.135000
83,0.151000


In [47]:
# Instantate the yield object w/ the defined parameters
ye = Yield(E0=E0, srim_energies=SRIM_ENERGIES, srim_ranges=SRIM_RANGES,
           target_thickness=TARGET_THICKNESS, dE=DE, density=DENSITY,
           molecular_weight=MOLECULAR_WEIGHT, projectile_intensity=PROJECTILE_INTENSITY, t_irrad=T_IRRAD)

#### Naming Convention for Cross Section Data
Filenames are parsed to infer the isotope name:

- If the name contains underscores, the **last underscore-separated token** is used.  
  *Example:* `NatGd_P_X_154m1Tb.csv` → `154m1Tb`  
- If there are no underscores, the **last alphanumeric sequence** is used.  
  *Example:* `NatGdPX155Tb.csv` → `NatGdPX155Tb`

The inferred name is then checked against the keys in the `isotope_half_lives` dictionary — files whose names don’t match a known isotope are skipped with a warning.

In [48]:
# Populate the reactions dictionary attribute of the Yield object
# Loads per-isotope cross-section data from CSVs in the given directory,
# with filenames following the naming convention described above
reactions = ye.load_reactions_from_csvs(
                directory="/content/example_data/yield/xs",
                isotope_half_lives=ISOTOPE_HALF_LIVES,
                filename_glob="NatGd_P_X_*.csv",  # or "*.csv"
                energy_col=0,
                xs_col=1,
                xs_units="mb",  # change if your CSVs are in barns/µb/nb/etc.
            )

In [49]:
# Calculate expected activity yields
results = ye.compute_activities_for_multiple_isotopes()

# Inspect available result fields for isotope 154m1Tb
# Per-slice activities, cross sections, and areal densities. Total expected yield
results['154m1Tb'].keys()

dict_keys(['slice_thicknesses', 'areal_densities', 'cross_sections', 'activities', 'total_activity'])

In [50]:
print(f"The expected 154m1Tb yield is {results['154m1Tb']['total_activity']} Bq")

The expected 154m1Tb yield is 3941297.5532004507 Bq


In [51]:
# Save results to Excel:
# - Each isotope’s per-slice information is written to its own sheet
# - A 'Summary' sheet provides an easy-to-read table of overall results
path = ye.save_results_to_excel("outputs/theoretical-yield-results.xlsx", include_summary=True)
print("Results saved to:", path)

Results saved to: /content/outputs/theoretical-yield-results.xlsx
